<h1 align="center"><font color="yellow">PGVector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

In [1]:
from dotenv import load_dotenv
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from psycopg.conninfo import make_conninfo
import psycopg

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connectionString = make_conninfo(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)

In [3]:
# Carregar documentos de amostra no banco de dados:

directory = './data'

# Percorra todos os arquivos no diretório, crie incorporações, salve-os no banco de dados:
for file_name in os.listdir(directory):
  file_path = os.path.join(directory, file_name)
  with open(file_path, 'r') as file:
      content = file.read()
      embedding_results = embeddings.embed_query(content)
      with psycopg.connect(connectionString) as conn:
        insert_query = "INSERT INTO documents (file_name, content, embedding) VALUES (%s, %s, %s)"
        insert_data = (file_name, content, embedding_results)
        conn.execute(insert_query, insert_data)
        conn.commit()

# NOTA: Você precisa criar o TABELA, lá no DB postgres, assim:
'''
CREATE TABLE documents (
    id serial PRIMARY KEY,
    file_name varchar,
    content text,
    embedding double precision[]
);
'''        


'\nCREATE TABLE documents (\n    id serial PRIMARY KEY,\n    file_name varchar,\n    content text,\n    embedding double precision[]\n);\n'

In [69]:
#search_text = "Como se chamava a tartaruga?"
search_text = "O que falou o presidente?"

# search do texto Embedding:
embedding_results = embeddings.embed_query(search_text)

print(type(embedding_results))
len(embedding_results)


<class 'list'>


1536

In [70]:
print(embedding_results)

[-0.014760998432233033, -0.0011918382486283715, 0.015622680396339185, 0.004661203645574282, 0.0015930201002985179, -0.005732062514410714, -0.007767631834466914, -0.00655628109809425, -0.00785504902753354, 0.012569327347577425, 0.04146067263602706, -0.0025897309103469924, -0.011451636924906377, -0.032044603168531804, 0.008598094470107976, -0.004305291287011513, 0.029097399468709248, -0.026150193906241615, 0.03159503142758996, -0.024227018452743415, 0.020455596169237325, -0.016684173885731235, 0.014236495273833267, 0.003980599188236373, -3.958647202966605e-05, -0.0026818310259664473, 0.006172270272891982, -0.008954006828670744, 0.018819648037107842, -0.020105927396970813, 0.036290577126099996, -0.013374813309727115, -0.018619838374467023, -0.0007528015118236036, -0.029072423260879145, -0.021517089658629373, -0.0035466357145432655, -0.0024086527050868455, -0.011270558952476865, 0.0017561467143884372, 0.014411329659966523, 0.006612477565711981, 0.02170441214867768, -0.0065063282167727755, 

In [73]:
# Encontre documentos com base na SIMILARIDADE de COSSENO com o texto de pesquisa Embedding:
# with psycopg.connect(connectionString) as conn:
#    select_query = "SELECT * FROM documents"
#    query_data = (f'{embedding_results}', 0.8, 10)
#    results = conn.execute(select_query, query_data).fetchall()
#    for result in results:
#      print(result) 



# Desta maneira não funcionou. Por isso fiz da maneira a seguir:

In [72]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Conectando ao banco de dados
with psycopg.connect(connectionString) as conn:
    cursor = conn.cursor()

    # Selecionar todos os documentos e seus embeddings da tabela
    select_documents_query = "SELECT id, file_name, content, embedding FROM documents;"
    cursor.execute(select_documents_query)
    results = cursor.fetchall()

    similar_documents = []

    for doc_id, doc_file_name, doc_content, doc_embedding in results:
        embedding_doc_DB = np.array(doc_embedding)  # Substitua pelo nome correto da coluna de embeddings
        similarity = cosine_similarity([embedding_results], [embedding_doc_DB])[0][0]

        if similarity >= 0.70:
            similar_documents.append((doc_id, doc_file_name, doc_content, similarity))

    # Ordenar os documentos pela similaridade de cosseno
    similar_documents.sort(key=lambda x: x[3], reverse=True)

    # Imprimir os documentos e suas similaridades
    for doc_id, doc_file_name, doc_content, similarity in similar_documents:
        print(f"Similaridade: {similarity:.4f}")
        print(f"Documento ID: {doc_id}")
        print("Nome do arquivo:", doc_file_name)
        print("Conteúdo:", doc_content)
        print("=" * 50)


Similaridade: 0.8146
Documento ID: 1
Nome do arquivo: RESUMINHO_estado_da_Uniao.txt
Conteúdo: Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.

No ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.

Hoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.

Com um dever para com o outro, para com o povo americano, para com a Constituição.

E com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.

Seis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.

Ele pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.

Ele conheceu o povo ucraniano.

Do p